# Trabajo Práctico - CEIA Vision por Computadora II

Alumno: Alianak, Juan Pablo

En este trabajo analizaremos el dataset Fruit. El dataset cuenta 3155 imagenes de 11 clases diferentes de frutas. 

El objetivo final será modelar un clasificador con redes convolucionales que sea capaz de, dada una imagen dentro de estas clases, predecir a cual de ellas pertenece.

Las noteboos estan divididas en 3 partes:

- VCP2-TP-Fruit

  En esta notebook implementaremos distintos modelos con y sin Data Aumentation para observar el comportamiento de cada uno de ellos.

- VCP2-TP-Fruit-Transfer_learning

  En esta notebook, con el modelo elegido en base al punto anterior, impplementaremos transfer lerning y evaluaremos resultados.

- VCP2-TP-Fruit-Optimization

  En esta notebook haremos optimizacion de algunos hiperparametros tomando como modelo el generado con trasfer learning.


In [1]:
!pip install ray[tune]
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Importamos las librerias necesarias

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchsummary

import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset

from google.colab.patches import cv2_imshow 
import cv2
import pandas as pd

import torchmetrics

import ray
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter

import os
import torch.optim as optim
from filelock import FileLock
from torch.utils.data import random_split

from torchvision.models.inception import Inception_V3_Weights

### Montamos el drive para acceder a los datos

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


### Carga de los nombres de las imagenes y de las clases

In [4]:
csv_train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/CEIA/VPC2/TP/Fruit_multiclass/train/classes.csv')
csv_valid = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/CEIA/VPC2/TP/Fruit_multiclass/valid/classes.csv')
csv_test  = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/CEIA/VPC2/TP/Fruit_multiclass/test/classes.csv')

In [5]:
class ImageDataset(Dataset):
  def __init__(self,csv,img_folder,transform):
    self.csv=csv
    self.transform=transform
    self.img_folder=img_folder
    self.image_names=self.csv[:]['filename']
    #self.labels=np.array(self.csv.drop(['filename'], axis=1), dtype=float)
    self.class2index = {'Apple':0, 'Banana':1, 'Coconut':2, 'Dragon':3, 'Grape':4, 'Mango':5, 'Orange':6, 'Papaya':7, 'Pineapple':8, 'Star_Fruit':9, 'Strawberry':10}

  #The __len__ function returns the number of samples in our dataset.
  def __len__(self):
    return len(self.image_names)
 
  def __getitem__(self,index):
     
    image=cv2.imread(self.img_folder+self.image_names.iloc[index])
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
 
    image=self.transform(image)
    targets = self.class2index[self.csv.loc[index]['label']]

    return image, targets

In [6]:
weights = Inception_V3_Weights.IMAGENET1K_V1

transf = weights.transforms()
Crop_size = transf.crop_size
Resize_size = transf.resize_size
Mean = transf.mean
Std = transf.std

In [7]:
def load_data():
  data_transforms = torchvision.transforms.Compose([
                          transforms.ToPILImage(),
                          torchvision.transforms.CenterCrop(size=(Crop_size[0], Crop_size[0])),
                          torchvision.transforms.Resize(size=(Resize_size[0], Resize_size[0])),
                          torchvision.transforms.ToTensor(),
                          transforms.Normalize(mean=Mean, std=Std)
                        ])

  train_set = ImageDataset(csv=csv_train,img_folder='/content/drive/MyDrive/Colab_Notebooks/CEIA/VPC2/TP/Fruit_multiclass/train/',transform=data_transforms)
  valid_set = ImageDataset(csv=csv_valid,img_folder='/content/drive/MyDrive/Colab_Notebooks/CEIA/VPC2/TP/Fruit_multiclass/valid/',transform=data_transforms)
  test_set  = ImageDataset(csv=csv_test,img_folder='/content/drive/MyDrive/Colab_Notebooks/CEIA/VPC2/TP/Fruit_multiclass/test/',transform=data_transforms)

  #train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
  #valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=32, shuffle=True)
  #test_loader  = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)

  return train_set, test_set

### Implementacion de los modelos

In [8]:
def Net():
  
  weights = Inception_V3_Weights.IMAGENET1K_V1

  model = torchvision.models.inception_v3(weights=weights)

  for param in model.parameters():
      param.requires_grad = False

  last_layer_in_features = model.fc.in_features
  model.fc = nn.Linear(last_layer_in_features, 11)

  return model

In [9]:
def train_inceptionv3(config):
  
  model = Net()

  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda:0"
      if torch.cuda.device_count() > 1:
          model = nn.DataParallel(model)
  model.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

  # To restore a checkpoint, use `session.get_checkpoint()`.
  loaded_checkpoint = session.get_checkpoint()
  
  if loaded_checkpoint:
      with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
         model_state, optimizer_state = torch.load(os.path.join(loaded_checkpoint_dir, "checkpoint.pt"))
      model.load_state_dict(model_state)
      optimizer.load_state_dict(optimizer_state)

  #data_dir = os.path.abspath("./data")
  trainset, testset = load_data()

  test_abs = int(len(trainset) * 0.8)
  train_subset, val_subset = random_split(trainset, [test_abs, len(trainset) - test_abs])

  trainloader = torch.utils.data.DataLoader(
      train_subset,
      batch_size=int(config["batch_size"]),
      shuffle=True,
      num_workers=8)
  valloader = torch.utils.data.DataLoader(
      val_subset,
      batch_size=int(config["batch_size"]),
      shuffle=True,
      num_workers=8)

  for epoch in range(2):  # loop over the dataset multiple times
      #model.train()
      running_loss = 0.0
      epoch_steps = 0
      for i, data in enumerate(trainloader, 0):
          #print('Batch train',i)
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs , _ = model(inputs)

          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          epoch_steps += 1
          if i % 2000 == 1999:  # print every 2000 mini-batches
              print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                              running_loss / epoch_steps))
              running_loss = 0.0

      # Validation loss
      val_loss = 0.0
      val_steps = 0
      total = 0
      correct = 0
      for i, data in enumerate(valloader, 0):
          with torch.no_grad():
              #model.eval()
              #print('Batch eval',i)
              inputs, labels = data
              inputs, labels = inputs.to(device), labels.to(device)
              outputs , _ = model(inputs)
              _, predicted = torch.max(outputs.data, 1)

              total += labels.size(0)
              correct += (predicted == labels).sum().item()

              loss = criterion(outputs, labels)
              val_loss += loss.cpu().numpy()
              val_steps += 1

      # Here we save a checkpoint. It is automatically registered with
      # Ray Tune and can be accessed through `session.get_checkpoint()`
      # API in future iterations.
      #print('pre save', epoch)
      #os.makedirs('/content/drive/MyDrive/Colab_notebooks/CEIA/VCP2/TP/Fruit_multiclass/Models/Result', exist_ok=True)
      torch.save((model.state_dict(), optimizer.state_dict()), '/content/drive/MyDrive/Colab_notebooks/CEIA/VPC2/TP/Fruit_multiclass/Models/Result/checkpoint.pt')
      checkpoint = Checkpoint.from_directory('/content/drive/MyDrive/Colab_notebooks/CEIA/VPC2/TP/Fruit_multiclass/Models/Result')
      session.report({"loss": (val_loss / val_steps), "accuracy": correct / total}, checkpoint=checkpoint)
  print("Finished Training")

In [10]:
def test_best_model(best_result):
    #best_trained_model = Net(best_result.config["l1"], best_result.config["l2"])
    best_trained_model = Net()
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs,_ = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


    print("Best trial test set accuracy: {}".format(correct / total))

In [11]:
num_samples=2
max_num_epochs=10
gpus_per_trial=0

config = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([16, 32, 64])
}

scheduler = ASHAScheduler(
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2
    )

reporter = CLIReporter(
    # parameter_columns=["l1", "l2", "lr", "batch_size"],
    metric_columns=["loss", "accuracy", "training_iteration"])
   
tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(train_inceptionv3),
        resources={"cpu": 2, "gpu": gpus_per_trial}
    ),
    tune_config=tune.TuneConfig(
        metric="loss",
        mode="min",
        scheduler=scheduler,
        num_samples=num_samples,
            
    ),
    param_space=config
)

results = tuner.fit()
   
best_result = results.get_best_result("loss", "min")

print("Best trial config: {}".format(best_result.config))
print("Best trial final validation loss: {}".format(
    best_result.metrics["loss"]))
print("Best trial final validation accuracy: {}".format(
    best_result.metrics["accuracy"]))

test_best_model(best_result)

2022-10-13 11:31:08,004	INFO worker.py:1518 -- Started a local Ray instance.
2022-10-13 11:31:09,995	WARNING function_trainable.py:620 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,batch_size,lr,iter,total time (s),loss,accuracy
train_inceptionv3_890f6_00000,TERMINATED,172.28.0.2:1389,32,0.0483239,2,1726.85,2.63797,0.67374
train_inceptionv3_890f6_00001,TERMINATED,172.28.0.2:1389,32,0.00063682,2,1734.95,1.42445,0.681698


(train_inceptionv3 pid=1389) /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
(train_inceptionv3 pid=1389)   cpuset_checked))


(train_inceptionv3 pid=1389) pre save 0
Result for train_inceptionv3_890f6_00000:
  accuracy: 0.5649867374005305
  date: 2022-10-13_11-45-42
  done: false
  experiment_id: d6515df668c040b596ad40b43bde97ac
  hostname: c2b40ee7fc0c
  iterations_since_restore: 1
  loss: 4.21274596452713
  node_ip: 172.28.0.2
  pid: 1389
  should_checkpoint: true
  time_since_restore: 867.909773349762
  time_this_iter_s: 867.909773349762
  time_total_s: 867.909773349762
  timestamp: 1665661542
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 890f6_00000
  warmup_time: 0.004838228225708008
  
(train_inceptionv3 pid=1389) pre save 1
Result for train_inceptionv3_890f6_00000:
  accuracy: 0.6737400530503979
  date: 2022-10-13_12-00-01
  done: false
  experiment_id: d6515df668c040b596ad40b43bde97ac
  hostname: c2b40ee7fc0c
  iterations_since_restore: 2
  loss: 2.6379695435365043
  node_ip: 172.28.0.2
  pid: 1389
  should_checkpoint: true
  time_since_restore: 1726.8505330085754
  time_this_iter_s

2022-10-13 12:28:57,012	INFO tune.py:759 -- Total run time: 3467.02 seconds (3466.62 seconds for the tuning loop).


Best trial config: {'lr': 0.0006368200672277239, 'batch_size': 32}
Best trial final validation loss: 1.4244477252165477
Best trial final validation accuracy: 0.6816976127320955
Best trial test set accuracy: 0.5107913669064749
